In [2]:
import sys
import numpy as np
sys.path.append('../datasets')
from datasets.manager import IMDBBinary, DD
import torch 
import itertools
from tqdm import tqdm

#from utils.utils import visualise_graph, get_adjacency_and_features
from utils.utils import get_adjacency_and_features, create_batch_from_loader
#from src.gnn import GNNClassifier

from datasets.dataset import *
from train import Training

import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

import random

import numpy as np
import time
import statistics
from tqdm import tqdm

import torch
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

from src.models import GCN, GAT, GraphDenseNet, GraphSAGE, GIN
from datasets.dataloader import DataLoader

In [3]:
params = {"model_type": "GAT",  # "GCN", "GAT", "GIN", "GraphSAGE"
               "n_graph_subsampling": 0, # the number of running graph subsampling each train graph data run subsampling 5 times: increasing graph data 5 times
               "graph_node_subsampling": True, # TRUE: removing node randomly to subsampling and augmentation of graph dataset \n'+
                # FALSE: removing edge randomly to subsampling and augmentation of graph dataset
               "graph_subsampling_rate": 0.2, # graph subsampling rate
               "dataset": "DD", 
               "pooling_type": "mean", 
               "seed": 42,
               "n_folds": 10, 
               "cuda": True, 
               "lr": 0.001, 
               "epochs": 50, 
               "weight_decay":5e-4,
               "batch_size": 32, 
               "dropout": 0, # dropout rate of layer
               "num_lay": 5, 
               "num_agg_layer": 2, # the number of graph aggregation layers
               "hidden_agg_lay_size": 64, # size of hidden graph aggregation layer
               "fc_hidden_size": 128, # size of fully-connected layer after readout
               "threads":10, # how many subprocesses to use for data loading
               "random_walk":True,
               "walk_length": 20, # walk length of random walk, 
               "num_walk": 10, # num of random walk
               "p": 0.65, # Possibility to return to the previous vertex, how well you navigate around
               "q": 0.35, # Possibility of moving away from the previous vertex, how well you are exploring new places
               "print_logger": 10,  # printing rate
               "eps":0.0, # for GIN only
               }

In [4]:
class Train:
    def __init__(self, params, dataset):
      self.params = params 
      self.x_dataset, self.y_dataset = dataset.dataset.get_data(), dataset.dataset.get_targets()

      if self.params["cuda"] and torch.cuda.is_available():
          self.device = "cuda:0"
      else: 
          self.device = "cpu"

      self.model = self.get_model()
      self.loss_fn = F.cross_entropy 

    
    def get_model(self):
        
        #adjacency, features = get_adjacency_and_features(self.dataset[0])
      
        if self.params["model_type"] == 'GCN':
          
            model = GCN(n_feat = self.x_dataset[0].x.shape[1],  # [N, F] → F
                    n_class=2,
                    n_layer=self.params['num_agg_layer'],
                    agg_hidden=self.params['hidden_agg_lay_size'],
                    fc_hidden=self.params['fc_hidden_size'],
                    dropout=self.params['dropout'],
                    pool_type=self.params['pooling_type'],
                    device=self.device).to(self.device)
            
        elif self.params["model_type"] == 'GAT':
            
            model = GAT(n_feat=self.x_dataset[0].x.shape[1],
                    n_class=2,
                    n_layer=self.params['num_agg_layer'],
                    agg_hidden=self.params['hidden_agg_lay_size'],
                    fc_hidden=self.params['fc_hidden_size'],
                    dropout=self.params['dropout'],
                    pool_type=self.params['pooling_type'],
                    device=self.device).to(self.device)
            
        elif self.params["model_type"] == 'GraphSAGE':
            
            model = GraphDenseNet(n_feat=self.x_dataset[0].x.shape[1],
                    n_class=2,
                    n_layer=self.params['num_agg_layer'],
                    agg_hidden=self.params['hidden_agg_lay_size'],
                    fc_hidden=self.params['fc_hidden_size'],
                    dropout=self.params['dropout'],
                    pool_type=self.params["pooling_type"],
                    device=self.device).to(self.device)
            
        elif self.params["model_type"] == 'GIN':
            
            model = GIN(n_feat=self.x_dataset[0].x.shape[1],
                    n_class=2,
                    n_layer=self.params['num_agg_layer'],
                    agg_hidden=self.params['hidden_agg_lay_size'],
                    fc_hidden=self.params['fc_hidden_size'],
                    dropout=self.params['dropout'],
                    pool_type=self.params["pooling_type"],
                    device=self.device).to(self.device)
            
        return model


    def loaders_train_test_setup(self):
        # On crée un "faux" DataLoader qui donne juste les indices, un par un (pas de batch)
        loader = torch.utils.data.DataLoader(
            range(len(self.x_dataset)),
            batch_size=1,  # important : batch de 1
            shuffle=True,
            num_workers=0,
            pin_memory=True,
            drop_last=False,
            collate_fn=lambda x: x  # x est une liste d'indices (longueur 1 ici)
        )

        # Comptage des paramètres
        c = sum(p.numel() for p in self.model.parameters() if p.requires_grad)
        print('N trainable parameters:', c)

        optimizer = optim.Adam(
            filter(lambda p: p.requires_grad, self.model.parameters()),
            lr=self.params["lr"],
            weight_decay=self.params["weight_decay"],
            betas=(0.5, 0.999)
        )

        scheduler = lr_scheduler.MultiStepLR(optimizer, [20, 30], gamma=0.1)

        return loader, optimizer, scheduler


    def train(self, train_loader, optimizer, scheduler, epoch):
        self.model.train()
        train_loss, n_samples = 0, 0
        total_time_iter = 0
        start = time.time()

        for batch_idx, data_batch in enumerate(train_loader):
            # data_batch est une liste d'indices, typiquement [idx]
            idx = data_batch[0]

            x = self.x_dataset[idx]
            y = self.y_dataset[idx]

            A, f = get_adjacency_and_features(x)
            A = A.to(self.device)
            f = f.to(self.device)
            y = torch.tensor([y], device=self.device)  # CORRECTION ICI


            optimizer.zero_grad()

            output = self.model(f, A)  # pas de batching
            loss = self.loss_fn(output.unsqueeze(0), y)  # output est [C], y est [1]

            loss.backward()
            optimizer.step()

            # Stat tracking
            time_iter = time.time() - start
            total_time_iter += time_iter
            train_loss += loss.item()
            n_samples += 1

            if batch_idx % self.params["print_logger"] == 0 or batch_idx == len(train_loader) - 1:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} (avg: {:.6f}) \tsec/iter: {:.4f}'.format(
                    epoch, n_samples, len(train_loader.dataset),
                    100. * (batch_idx + 1) / len(train_loader),
                    loss.item(), train_loss / n_samples, time_iter / (batch_idx + 1)
                ))

            start = time.time()  # reset pour la prochaine itération

        scheduler.step()
        return total_time_iter / (len(train_loader) + 1)


    def evaluate(self, test_loader):
        self.model.eval()
        correct, n_samples = 0, 0

        with torch.no_grad():
            for batch_idx, data_batch in enumerate(test_loader):
                idx = data_batch[0]  # data_batch est une liste d’indices
                x = self.x_dataset[idx]
                y = self.y_dataset[idx]

                A, f = get_adjacency_and_features(x)
                A = A.to(self.device)
                f = f.to(self.device)
                y = torch.tensor([y], device=self.device)  # CORRECTION ICI


                output = self.model(f, A)

                # Prédiction (binaire ou multi-class)
                if output.shape[-1] == 1:
                    pred = (torch.sigmoid(output) > 0.5).long()
                else:
                    pred = output.argmax(dim=-1)

                correct += (pred == y).sum().item()
                n_samples += 1

        acc = 100. * correct / n_samples
        print(f'Test set (epoch {self.params["epochs"]}): Accuracy: {correct}/{n_samples} ({acc:.2f}%)\n')

        return acc

    def fit(self): 
        loader, optimizer, scheduler = self.loaders_train_test_setup()
        total_time = 0

        best_acc = 0
        patience_counter = 0
        patience = self.params.get("early_stopping_patience", 5)

        for epoch in tqdm(range(self.params["epochs"]), desc="Epochs", position=1, leave=False):
            total_time_iter = self.train(loader, optimizer, scheduler, epoch)
            total_time += total_time_iter
            acc = self.evaluate(loader)  # Même loader pour train/test si pas de split

            if acc > best_acc:
                best_acc = acc
                patience_counter = 0
            else:
                patience_counter += 1

            if patience_counter >= patience:
                print(f"Early stopping triggered after {epoch + 1} epochs.")
                break

        print(f'Best Accuracy: {best_acc:.2f}%')
        print(f'Average training time per epoch: {total_time / (epoch + 1):.2f} seconds')

        result_list = [self.params["dataset"], self.params["dataset"], best_acc]
        return result_list


In [5]:
IMDB = IMDBBinary()
DD = DD()

In [6]:
trainer = Train(params, DD)

[DEBUG] GAT: n_feat=89, agg_hidden=64
[DEBUG] GraphAttentionLayer: W shape = (89, 64)
[DEBUG] GraphAttentionLayer: W shape = (64, 64)


In [6]:
trainer.fit()

N trainable parameters: 26946


RuntimeError: mat1 and mat2 shapes cannot be multiplied (436x89 and 64x64)

In [9]:
model = trainer.model
x_dataset = trainer.x_dataset
x0, x1 = x_dataset[0], x_dataset[1]
A0, f0 = get_adjacency_and_features(x0)
A1, f1 = get_adjacency_and_features(x1)
print(A0.shape)
print(f0.shape)

# model(f0, A0)

weight = torch.nn.Parameter(torch.FloatTensor(89, 64))
h = torch.mm(x0, weight)

torch.Size([327, 327])
torch.Size([327, 89])


TypeError: mm(): argument 'input' (position 1) must be Tensor, not GraphData